In [1]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Para el dataset final unificado necesito:

* CBC
* Columna de tiempo de viaje
* Actas de la FCEN
* Datos personales de alumnos de la FCEN
* Hace cuanto hizo el CBC

In [ ]:
# columnas del CBC
cbc = pd.read_csv('../../../assets/silver/data_utilizada/cbc_final.csv')
cbc['Dni'] = cbc['Dni'].astype(str)
cbc.head(5)

In [ ]:
actas = pd.read_csv('../../../assets/silver/data_utilizada/actas_entrenamiento.csv')
actas.head(5)

In [ ]:
resumen_semestre = pd.read_csv('../../../assets/silver/data_utilizada/resumen_personas_por_semestre.csv')
resumen_semestre.head(5)

In [ ]:
# datos personales
datos_personales = pd.read_csv('../../../assets/silver/data_utilizada/personas_con_target.csv')
datos_personales.head(5)

In [ ]:
# tiempo de viaje
tiempo_viaje = pd.read_csv('../../../assets/silver/data_utilizada/tiempo_viaje_manana.csv')
tiempo_viaje.head(5)

In [ ]:
anios_cbc = pd.read_csv('../../../assets/silver/data_utilizada/tiempo_desde_cbc.csv')
anios_cbc['dni'] = anios_cbc['dni'].astype(str)
anios_cbc.head(5)

Veamos que todos tengan el tipo de dato para poder hacer la unificación y la cantidad de datos correctos.

In [ ]:
def data_basica(df, dni):
    print(df.columns)
    print(df.shape)
    print(df[f'{dni}'].nunique())
    print(df.dtypes)

In [ ]:
data_basica(cbc, 'Dni')

In [ ]:
data_basica(actas, 'Dni')

In [ ]:
data_basica(resumen_semestre, 'dni')

In [ ]:
data_basica(datos_personales, 'dni')

In [ ]:
data_basica(tiempo_viaje, 'dni')

In [ ]:
data_basica(anios_cbc, 'dni')

Todo parece estar bien el único que falta corregir es resumen_semestre

In [ ]:
resumen_semestre.drop(columns=['total_actividad'], inplace=True)
resumen_semestre

In [ ]:
resumen_semestre = resumen_semestre[resumen_semestre['semestre_relativo'] != 4]

In [ ]:
resumen_semestre['dni'].nunique()

In [ ]:
# Hacemos melt para que quede en formato largo (si no lo está ya)
resumen_semestre_melt = pd.melt(
    resumen_semestre, 
    id_vars=['dni', 'semestre_relativo'], 
    value_vars=['inscripciones', 'tp_aprobados', 'finales_inscriptos']
)

In [ ]:
# Ahora pivotamos para que cada variable quede con su sufijo de semestre
resumen_semestre_pivot = resumen_semestre_melt.pivot_table(
    index='dni', 
    columns=['variable', 'semestre_relativo'], 
    values='value'
)

# Acomodamos los nombres de las columnas
resumen_semestre_pivot.columns = [f'{var}_{sem}' for var, sem in resumen_semestre_pivot.columns]
resumen_semestre_pivot = resumen_semestre_pivot.reset_index()

In [ ]:
#resumen_semestre_pivot.to_csv('../../../assets/silver/data_utilizada/resumen_personas_por_semestre_pivot.csv', index=False)

In [8]:
resumen_semestre = pd.read_csv('../../../assets/silver/data_utilizada/resumen_personas_por_semestre_pivot.csv')

In [ ]:
data_basica(resumen_semestre, 'dni')

## JUNTAMOS TODO

In [ ]:
# Unimos datos personales con resumen_semestre
datos_personales.columns

In [ ]:
datos_personales.drop(columns=['codigo_postal', 'localidad', 'departamento', 'provincia', 'pais', 'dominio_email', 'carrera_principal', 'acceso_a_internet', 'grupo_edad'], inplace=True)

In [ ]:
cbc.rename(columns={'Dni': 'dni'}, inplace=True)

In [ ]:
dataset_gold = datos_personales.merge(cbc, on='dni', how='left')
dataset_gold.shape

In [ ]:
resumen_semestre.columns

In [ ]:
dataset_gold = dataset_gold.merge(resumen_semestre, on='dni', how='left')
dataset_gold.shape

In [ ]:
actas.columns

In [ ]:
actas.rename(columns={'Dni': 'dni'}, inplace=True)

In [ ]:
dataset_gold = dataset_gold.merge(actas, on='dni', how='left')
dataset_gold.shape

In [ ]:
dataset_gold = dataset_gold.merge(tiempo_viaje, on='dni', how='left')
dataset_gold.shape

In [ ]:
anios_cbc.columns

In [ ]:
dataset_gold = dataset_gold.merge(anios_cbc, on='dni', how='left')
dataset_gold.shape

In [ ]:
dataset_gold

In [ ]:
dataset_gold.to_csv('../../../assets/gold/dataset_gold_v1.csv', index=False)

In [ ]:
dataset_gold.columns

In [ ]:
dataset_gold_test = dataset_gold.drop(columns = ['dni'])

In [ ]:
# Verifica si hay filas duplicadas
duplicados = dataset_gold_test.duplicated()

# Mostrar cuántas filas duplicadas hay
print(f"Cantidad de filas duplicadas: {duplicados.sum()}")

# Opcional: mostrar las filas duplicadas
filas_duplicadas = dataset_gold_test[duplicados]
print(filas_duplicadas)

In [ ]:
dataset_gold_test = dataset_gold_test.drop(columns = ['nacionalidad'])

In [ ]:
# Verifica si hay filas duplicadas
duplicados = dataset_gold_test.duplicated()

# Mostrar cuántas filas duplicadas hay
print(f"Cantidad de filas duplicadas: {duplicados.sum()}")

# Opcional: mostrar las filas duplicadas
filas_duplicadas = dataset_gold_test[duplicados]
print(filas_duplicadas)

In [ ]:
dataset_gold_test = dataset_gold_test.drop(columns = ['tiempo_desde_cbc'])
# Verifica si hay filas duplicadas
duplicados = dataset_gold_test.duplicated()

# Mostrar cuántas filas duplicadas hay
print(f"Cantidad de filas duplicadas: {duplicados.sum()}")

# Opcional: mostrar las filas duplicadas
filas_duplicadas = dataset_gold_test[duplicados]
print(filas_duplicadas)